# COVID-19 Prediction with ARIMA

## 1. Data Pre-processing

We first define some helper classes and functions to aid in our data pre-processing. Using the `numpy` and `pandas` library, we provide ourselves with some useful and needed functionalities such as parsing the input data into a dataframe, dropping unneeded features and accessing certain needed features from the input data.

The following code blocks in 1.1 should be run first.

### 1.1 Constants, Helper Functions, and Helper Classes

In [1]:
train_data_dir = 'data/train'
submissions_dir = 'data/submission'

In [2]:
import os
import numpy as np
import pandas as pd

In [3]:
# https://gist.github.com/rogerallen/1583593
# ^ Use this in case we need state code translation

from sklearn.preprocessing import StandardScaler

import math
import matplotlib.pyplot as plt
class utils:
    def __init__(self):
        pass
    
    def split(self, dataframe, test=0.2):
        size = dataframe.shape
        train_size = size[0] - math.floor(size[0] * test)
        return dataframe[:train_size], dataframe[train_size:]
    
class CoreData(object):
    def __init__(self, data_path = None, debug=False):
        if not data_path:
            raise Exception("Input file path!")
        self.data_path = data_path
        self.df = None
        self.states = []
        self.state_df = {}
        self.debug = debug
        
    def load(self, normalize=True):
        '''
        DataProcessor.load()
        Pre-load data state-by-state to a dictionary.
        '''
        # Change this line to modify dropped data series.
        dropped_col = ['ID', 'Province_State', 'Date', 'Incident_Rate', 'Recovered', 'People_Tested', 'People_Hospitalized', 'Mortality_Rate', 'Testing_Rate', 'Hospitalization_Rate' ]
        
        self.df = pd.read_csv(train_data_path)
        self.states = list(np.unique(self.df['Province_State']))
        self.state_df = dict.fromkeys(self.states, None)
        self.mean = dict.fromkeys(self.states, None)
        self.std = dict.fromkeys(self.states, None)
        
        for s in self.states:
            df_filter = self.df['Province_State'] == s
            tmp_state_df = self.df[df_filter]
            self.state_df[s] = tmp_state_df.drop(dropped_col, 1)
            
            # Normalize
            if normalize:
                self.mean[s] = self.state_df[s].mean()
                self.std[s] = self.state_df[s].std()
                self.state_df[s] = (self.state_df[s] - self.mean[s])/self.std[s]
    
    def access(self, state=None):
        if not state:
            raise Exception('Enter state name! i.e. self.access("California")')
        elif state not in self.states:
            raise Exception('Check your spelling of the state.')
        return self.state_df[state]
    
    def access_split(self, state=None, test_portion=0.2):
        df = self.access(state)
        return utils().split(dataframe=df, test=test_portion)


## 2. Round 1 Forecasting

In this section, we build and train an ARIMA model to predict COVID-19 confirmed cases and deaths for each state between Sept 1 and Sept 26, using data from April 12 to August 31. Each state will have its own model and its own set of hyperparameters, as the trends can be wildly different for different states. Data for round 1 is located in the `data/round1/` directory.

### 2.1 Load Round 1 Data 

In [4]:
import math
from pandas.plotting import lag_plot
from datetime import datetime
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
from pandas.plotting import autocorrelation_plot
import warnings
warnings.filterwarnings('ignore')

In [5]:
train_data_path = os.path.join(train_data_dir, "train.csv")
dp = CoreData(train_data_path)
dp.load(False)

### 2.2 Forecasting Confirmed Cases

In [6]:
N_test = 26

# states in outliers1 are predited to have lower value than the ground truth from our cross-validation reults.
# states in outliers2 are predited to have higher value than the ground truth from our cross-validation reults.
outliers1 = ['West Virginia', 'North Dakota','Wisconsin','Utah', 'Wyoming', 'Montana']
outliers2 = ['Alabama', 'Vermont']

result_confirmed = {}
states = dp.states
for state in states:
    print(f"Processing: {state}...")
    training_data, test_data = dp.access_split(state, 0)
    data = dp.access(state)
    training = training_data['Confirmed'].values
    testing = test_data['Confirmed'].values

    # Use the differences between two days to train our model
    temp_data = []
    for i in range(1, len(training)):
        temp_data.append(training[i] - training[i-1])
    training = temp_data

    #Starting training
    history = [x for x in training]
    model_predictions = []
    for time in range(N_test):
        # Choose parameters (p,d,q) for ARIMA depended on the population of the state
        # and use larger lag 8 for long-period prediction
        try:
            if sum(training) / len(training) > 1500:
                model = ARIMA(history, order=(8,2,1))
            elif time < 14:
                model = ARIMA(history, order=(1,1,1))
            else:
                model = ARIMA(history, order=(8,2,1))
            model_fit = model.fit(disp=0)
        except:
            # In case the model cannot converge with the larger lag parameters.
            model = ARIMA(history, order=(1,1,0))
            model_fit = model.fit(disp=0)

        output = model_fit.forecast()
        yhat = output[0]
        # Force the result to at least increase by a small value to avoid predictions of all-zeros.
        if yhat[0] < 0.5:
            yhat[0]=0.5

        # Hawaii is a speacial case that cause very larger error, so it has differnet treatment.
        if state == 'Hawaii':
            if time < 14:
                yhat[0] = 150
            else:
                yhat[0] = 100
        
        model_predictions.append(yhat)
        true_test_value = yhat
        # Feedback the result to history for the next prediction
        history.append(true_test_value)
    
    # Process the results
    M = []
    for m in model_predictions:
        M.append(m.tolist()[0])
    
    # Restore the values from differences
    training.insert(0, training_data['Confirmed'].values[0])
    for i in range(1, len(training)):
        training[i] = training[i-1] + training[i]
    M[0] = training[-1] + M[0]
    for i in range(1, len(M)):
        M[i] = M[i-1] + M[i]
        if i >=16:
            # Process the outliers
            if state in outliers1:
                M[i] = M[i] + M[0] / 50
            elif state in outliers2:
                M[i] = M[i] - M[0] / 100
    print(f"Done: {state}...{M}")
    
    result_confirmed[state] = M


Processing: Alabama...
Done: Alabama...[127362.63423385199, 128647.87568384389, 129942.54513021321, 131244.98295818185, 132555.27982160114, 133873.43556191507, 135199.4483386472, 136533.31764166662, 137875.0405758347, 139224.62050414382, 140582.05728692655, 141947.3469314635, 143320.49264824044, 144701.4935842564, 146071.4078692138, 147444.41705205402, 147545.33633458367, 147648.86417452522, 147756.2766621162, 147867.69676413675, 147978.83014107883, 148092.11181934667, 148204.9855985609, 148319.951068223, 148436.51135029335, 148555.0977660603]
Processing: Alaska...
Done: Alaska...[5317.796618935081, 5380.494861761127, 5442.405750859091, 5504.900065780807, 5570.590669608626, 5637.905384675721, 5706.171996007348, 5775.10302437061, 5844.575474576718, 5914.53668322711, 5984.964060507105, 6055.84789310308, 6127.183973977094, 6198.9704637775485, 6271.258890947, 6344.0097848827845, 6417.212053513797, 6490.855749365532, 6564.955957504333, 6639.515344078827, 6714.540349604909, 6790.03702109718,

Done: Kentucky...[48810.338257390926, 49438.164400110836, 50051.02990530245, 50669.01990984483, 51290.36647728107, 51915.22382367401, 52543.57668897535, 53175.426049269336, 53810.77049920435, 54449.6095655475, 55091.942684840455, 55737.7693967022, 56387.089041249754, 57039.90146222647, 57704.32563234446, 58372.23384556098, 59044.025039904496, 59719.650949022, 60399.84984987456, 61084.790454182934, 61774.65240305093, 62470.91941485621, 63172.17172102673, 63878.26131748232, 64588.745104483925, 65304.07265423164]
Processing: Louisiana...
Done: Louisiana...[149048.75960578804, 149716.53532892186, 150454.28032688724, 151169.28965042328, 151894.87119051386, 152619.06026180187, 153346.15100638545, 154074.6149815851, 154804.9940496135, 155537.08431402818, 156270.96657048506, 157006.60889532065, 157744.03181467077, 158483.2237095588, 159205.21316653764, 159931.89984775535, 160659.56129873943, 161386.7055748621, 162114.33853816846, 162842.32140667186, 163571.3867175554, 164292.8855252963, 165017

Done: New York...[435367.39346621884, 435934.87041098456, 436458.336364308, 436937.7201423461, 437372.97418105183, 437764.1668165094, 438111.2569844712, 438414.2101865875, 438672.9993048512, 438887.7180901796, 439058.3480081513, 439184.8903507869, 439267.4103534072, 439305.8815829276, 439306.3815829276, 439306.8815829276, 439307.3815829276, 439307.8815829276, 439308.3815829276, 439308.8815829276, 439309.3815829276, 439309.8815829276, 439310.3815829276, 439310.8815829276, 439311.3815829276, 439311.8815829276]
Processing: North Carolina...
Done: North Carolina...[168890.22379472223, 170462.1254361788, 172044.16250532595, 173635.40761007293, 175235.9028628709, 176845.65029073233, 178464.64372739402, 180092.88589530432, 181730.3738161921, 183377.10964269855, 185033.09286607333, 186698.32253838074, 188372.80086521848, 190056.52568718267, 191748.3269207477, 193449.58804847137, 195160.08135445803, 196879.8726294956, 198609.02187920208, 200347.42613771622, 202094.85372262064, 203851.2333514489

Done: Washington...[75168.66429468441, 75650.7034042549, 76148.41561045861, 76645.0413674391, 77144.75365472322, 77646.51222410341, 78150.57583195562, 78656.87908232312, 79165.43834189333, 79676.24903242098, 80189.31202307888, 80704.62709173994, 81222.19373195348, 81742.01225026629, 82244.00679532801, 82752.50935788348, 83263.21372596484, 83774.20527247446, 84283.87247460833, 84795.83052160716, 85305.24066504987, 85810.08014482241, 86316.35570849956, 86822.81605130041, 87329.06753781, 87834.45485347406]
Processing: West Virginia...
Done: West Virginia...[10381.938243415154, 10516.289408898425, 10651.308287071572, 10787.060849473179, 10923.541602625643, 11060.751258301165, 11198.689934699405, 11337.35777303401, 11476.754865879555, 11616.881289035236, 11757.737118999023, 11899.322404501147, 12041.637178317078, 12184.681484701185, 12328.497753631413, 12473.066938946591, 12826.032611147668, 13179.753174280908, 13534.229699113803, 13889.47099718116, 14245.481110864554, 14602.266338808293, 1

### 2.3 Forecasting Deaths

In [7]:
N_test = 26

# states in outliers1 are predited to have lower value than the ground truth from our cross-validation reults.
# states in outliers2 are predited to have higher value than the ground truth from our cross-validation reults.
outliers1 = ['Missouri', 'Kansas', 'North Dakota', 'Montana', 'North Dakota', 'Ohio', 'Kentucky', 'Arizona', 'Florida', 'South Dakota', 'Utah']
outliers2 = ['Mississippi', 'Alabama', 'Idaho', 'Georgia', 'New York', 'Indiana', 'Oregon', 'Nebraska','Michigan','South Carolina']
result_death = {}
states = dp.states
for state in states:
    print(f"Processing: {state}...")
    training_data, test_data = dp.access_split(state, 0)
    data = dp.access(state)
    training = training_data['Deaths'].values
    testing = test_data['Deaths'].values

    # Use the differences between two days to train our model
    temp_data = []
    for i in range(1, len(training)):
        temp_data.append(training[i] - training[i-1])
    training = temp_data

    #Starting training
    history = [x for x in training]
    model_predictions = []
    for time in range(N_test):
        # Choose parameters (p,d,q) for ARIMA depended on the population of the state
        # and use larger lag 6 for long-period prediction
        try:
            if sum(training) / len(training) > 30:
                model = ARIMA(history, order=(6,2,1))
            elif time < 14:
                model = ARIMA(history, order=(1,1,1))
            else:
                model = ARIMA(history, order=(6,2,1))
            model_fit = model.fit(disp=0)
        except:
            # In case the model cannot converge with the larger lag parameters.
            model = ARIMA(history, order=(1,1,0))
            model_fit = model.fit(disp=0)
        output = model_fit.forecast()
        yhat = output[0]

        # Force the result to at least increase by a small value to avoid predictions of all-zeros.
        if yhat[0] < 0.5:
            yhat[0]=0.5

        # Hawaii and Vermont are speacial cases that cause very larger error, so they have differnet treatment.
        if state == 'Hawaii':
            if time < 18:
                if yhat[0] > 2:
                    yhat[0] = 2
            else:
                yhat[0] = 3
        if state == 'Vermont':
            yhat[0] = 0


        model_predictions.append(yhat)
        true_test_value = yhat

        # Feedback the result to history for the next prediction
        history.append(true_test_value)

    # Process the results
    M = []
    for m in model_predictions:
        M.append(m.tolist()[0])

    # Restore the values from differences
    training.insert(0, training_data['Deaths'].values[0])
    for i in range(1, len(training)):
        training[i] = training[i-1] + training[i]
    M[0] = training[-1] + M[0]
    for i in range(1, len(M)):
        # Process the outliers
        if state in outliers1:
            M[i] += M[i-1] + M[i] * 1.05
        elif state in outliers2:
            M[i] += M[i-1] + M[i] * 0.05
        else:
            M[i] = M[i-1] + M[i]
    print(f"Done: {state}...{M}")
    
    result_death[state] = M


Processing: Alabama...
Done: Alabama...[2203.308348036696, 2226.098553653819, 2249.06633905282, 2272.154376717411, 2295.3484271030798, 2318.64569426261, 2342.045196368017, 2365.546732850695, 2389.1502280508425, 2412.8556425939573, 2436.6630964233113, 2460.5725449869133, 2484.584118782906, 2508.696763726119, 2532.9784743446667, 2557.3823216779865, 2581.8961266996716, 2606.5210581599977, 2631.2511166094855, 2656.0912093317133, 2681.0444833802226, 2706.139103056403, 2731.3601913693087, 2756.6976436043606, 2782.150878180873, 2807.7152247676668]
Processing: Alaska...
Done: Alaska...[37.501993035178096, 38.001993035178096, 38.501993035178096, 39.001993035178096, 39.501993035178096, 40.002172690889175, 40.50636895984752, 41.01443125537033, 41.5263655381626, 42.04217289980235, 42.561844548416374, 43.085403533529934, 43.61281611735732, 44.144094108641234, 44.694468400539535, 45.251147741097896, 45.81519360311259, 46.388248758709594, 46.96975945269624, 47.55850041146752, 48.15418147462028, 48.76

Done: Louisiana...[4976.191025164207, 5000.925248032511, 5025.54102973484, 5049.831107920434, 5073.827512796659, 5097.525222317432, 5120.924962229692, 5144.026594898772, 5166.8301018175125, 5189.335473341844, 5211.542690025496, 5233.451754629045, 5255.06264540232, 5276.375370871337, 5300.928351518926, 5325.352023148931, 5350.410903144824, 5375.901972528301, 5400.770123824613, 5426.237142500643, 5451.984618200242, 5479.138027691954, 5506.806685293185, 5535.061294542894, 5563.867956793734, 5592.6045584023495]
Processing: Maine...
Done: Maine...[132.5, 133.0, 133.5, 134.0, 134.5, 135.0, 135.5, 136.0, 136.5, 137.0, 137.5, 138.0, 138.5, 139.0, 139.53921161826912, 140.0837933014866, 140.62845624995884, 141.18719776707923, 141.7532705272018, 142.33021035427223, 142.9047351129623, 143.50527515493712, 144.11805488679772, 144.73612313498174, 145.36944886921998, 146.0134671292122]
Processing: Maryland...
Done: Maryland...[3761.7568602569145, 3767.019418404185, 3772.3740713311045, 3777.34163897240

Done: North Dakota...[144.244183290127, 146.74726126861947, 149.26249993664325, 151.78339556527303, 154.31064793302437, 156.844173235671, 159.3839734440795, 161.93004177461515, 164.4823728188361, 167.0409613774463, 169.60580277291183, 172.17689228645222, 174.75422588756263, 177.33779949748154, 179.88636056375844, 182.44025030549113, 184.99791057038914, 187.5536872842686, 190.10981528519739, 192.65776146692397, 195.1999565202643, 197.7378662952887, 200.27341431379074, 202.8070229541967, 205.33550336965146, 207.85859473814386]
Processing: Ohio...
Done: Ohio...[4139.5, 4149.863898884657, 4154.8649640346175, 4162.0048141601665, 4189.323988792176, 4205.900854472534, 4227.512935926595, 4246.109548501981, 4265.798802520881, 4284.484271462509, 4303.235846293883, 4321.507504883347, 4339.577888516622, 4357.304803079776, 4374.760827464759, 4391.9089127400475, 4408.767988589174, 4425.3282827723315, 4441.594801698053, 4457.564999280199, 4473.2402507671095, 4488.619752392526, 4503.703915858116, 4518

Done: Wyoming...[37.5, 38.0, 38.5, 39.0, 39.5, 40.0, 40.5, 41.0, 41.5, 42.0, 42.5, 43.0, 43.5, 44.0, 44.5, 45.026231651609855, 45.55616737516828, 46.096982777037475, 46.64386196041513, 47.19520124517187, 47.752735793251354, 48.31987758942757, 48.89562062948412, 49.4804446758691, 50.07378023392268, 50.67470303271469]


### 2.4 Generate Submission Output File

In [8]:
forecastID = [x for x in range(N_test*50)]
deaths = []
confirmed = []

for i in range(N_test):
    for s in states:
        print(f"Day {i}/{N_test}, {s}")
        confirmed.append(result_confirmed[s][i])
        deaths.append(result_death[s][i])


Day 0/26, Alabama
Day 0/26, Alaska
Day 0/26, Arizona
Day 0/26, Arkansas
Day 0/26, California
Day 0/26, Colorado
Day 0/26, Connecticut
Day 0/26, Delaware
Day 0/26, Florida
Day 0/26, Georgia
Day 0/26, Hawaii
Day 0/26, Idaho
Day 0/26, Illinois
Day 0/26, Indiana
Day 0/26, Iowa
Day 0/26, Kansas
Day 0/26, Kentucky
Day 0/26, Louisiana
Day 0/26, Maine
Day 0/26, Maryland
Day 0/26, Massachusetts
Day 0/26, Michigan
Day 0/26, Minnesota
Day 0/26, Mississippi
Day 0/26, Missouri
Day 0/26, Montana
Day 0/26, Nebraska
Day 0/26, Nevada
Day 0/26, New Hampshire
Day 0/26, New Jersey
Day 0/26, New Mexico
Day 0/26, New York
Day 0/26, North Carolina
Day 0/26, North Dakota
Day 0/26, Ohio
Day 0/26, Oklahoma
Day 0/26, Oregon
Day 0/26, Pennsylvania
Day 0/26, Rhode Island
Day 0/26, South Carolina
Day 0/26, South Dakota
Day 0/26, Tennessee
Day 0/26, Texas
Day 0/26, Utah
Day 0/26, Vermont
Day 0/26, Virginia
Day 0/26, Washington
Day 0/26, West Virginia
Day 0/26, Wisconsin
Day 0/26, Wyoming
Day 1/26, Alabama
Day 1/26, 

In [9]:
round1_path = os.path.join(submissions_dir, 'Team4_Round1.csv')
final = pd.DataFrame(list(zip(forecastID, confirmed, deaths)), 
               columns =['ForecastID', 'Confirmed', 'Deaths']) 
final.to_csv(round1_path, index=False)

## 3. Round 2 Forecasting

In this section, we build and train an ARIMA model to predict COVID-19 confirmed cases and deaths for each state between Dec 7 and Dec 13, using data from April 12 to December 5. The data from April 12 to November 22 is given, and the data from November 23 to December 5 is obtained by crawling Johns Hopkins University's dataset for confirmed COVID-19 cases and deaths. Each state will have its own model and its own set of hyperparameters, with further tweaks from round 1, as the trends can be wildly different for different states. Data for round 1 is located in the `data/round2/` directory.

### 3.1 Load Round 2 Data

In [9]:
import math
from pandas.plotting import lag_plot
from datetime import datetime
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
from pandas.plotting import autocorrelation_plot
import warnings
warnings.filterwarnings('ignore')

In [10]:
train_data_path = os.path.join(train_data_dir, "train_round2.csv")
dp = CoreData(train_data_path)
dp.load(False)

### 3.2 Forecasting Confirmed Cases

In [11]:
N_test = 7

result_confirmed = {}
states = dp.states
for state in states:
    print(f"Processing: {state}...")
    training_data, test_data = dp.access_split(state, 0)
    data = dp.access(state)
    training = training_data['Confirmed'].values
    testing = test_data['Confirmed'].values

    # Use the differences between two days to train our model
    temp_data = []
    for i in range(1, len(training)):
        temp_data.append(training[i] - training[i-1])
    training = temp_data
    
    #Starting training
    history = [x for x in training]
    model_predictions = []
    for time in range(N_test):
        # Choose parameters (p,d,q) for ARIMA depended on the population of the state
        # and use larger lag 8 for long-period prediction
        try:
            if sum(training) / len(training) > 1500:
                model = ARIMA(history, order=(8,2,1))
            elif time < 14:
                model = ARIMA(history, order=(1,1,1))
            else:
                model = ARIMA(history, order=(8,2,1))
            model_fit = model.fit(disp=0)
        except:
            # In case the model cannot converge with the larger lag parameters.
            model = ARIMA(history, order=(1,1,0))
            model_fit = model.fit(disp=0)
        
        output = model_fit.forecast()
        yhat = output[0]
        # Force the result to at least increase by a small value to avoid predictions of all-zeros.
        if yhat[0] < 0.5:
            yhat[0]=0.5
        model_predictions.append(yhat)
        true_test_value = yhat

        # Feedback the result to history for the next prediction
        history.append(true_test_value)

    # Process the results
    M = []
    for m in model_predictions:
        M.append(m.tolist()[0])
    
    # Restore the values from differences
    training.insert(0, training_data['Confirmed'].values[0])
    for i in range(1, len(training)):
        training[i] = training[i-1] + training[i]
    M[0] = training[-1] + M[0]
    for i in range(1, len(M)):
        M[i] = M[i-1] + M[i]
    print(f"Done: {state}...{M}")
    
    result_confirmed[state] = M[-7:]

Processing: Alabama...
Done: Alabama...[270765.66579695867, 273913.8796063688, 277067.2217349944, 280231.7917991511, 283408.7043617718, 286598.16443212336, 289800.2069186907]
Processing: Alaska...
Done: Alaska...[37024.6648364356, 37772.67679436186, 38523.44865938673, 39277.360530716796, 40034.43064904476, 40794.66030019774, 41558.05013065874]
Processing: Arizona...
Done: Arizona...[364347.9467338932, 370109.31413161044, 375832.9999577312, 381593.078751198, 387373.9193127212, 393178.8235872269, 399007.0592470166]
Processing: Arkansas...
Done: Arkansas...[171745.98596079682, 173917.44149011455, 176012.5925977049, 178080.42245343694, 180141.57450630137, 182204.75072963882, 184273.61891767624]
Processing: California...
Done: California...[1363236.4814962118, 1389739.0190118572, 1417096.863298124, 1447309.701215487, 1479037.362354616, 1514064.1117877793, 1550171.970116632]
Processing: Colorado...
Done: Colorado...[262386.4185934277, 267409.76666780683, 272440.7068307425, 277487.3069017269,

### 3.3 Forecasting Deaths

In [12]:
N_test = 7

result_death = {}
states = dp.states
for state in states:
    print(f"Processing: {state}...")
    training_data, test_data = dp.access_split(state, 0)
    data = dp.access(state)
    training = training_data['Deaths'].values
    testing = test_data['Deaths'].values
    temp_data = []
    for i in range(1, len(training)):
        temp_data.append(training[i] - training[i-1])
    training = temp_data
    history = [x for x in training]
    model_predictions = []
    print(sum(training) / len(training))
    for time in range(N_test):
        # Choose parameters (p,d,q) for ARIMA depended on the population of the state
        # and use larger lag 6 for long-period prediction
        try:
            if sum(training) / len(training) > 30:
                model = ARIMA(history, order=(6,2,1))
            elif time < 14:
                model = ARIMA(history, order=(1,1,1))
            else:
                model = ARIMA(history, order=(6,2,1))
            model_fit = model.fit(disp=0)
        except:
            # In case the model cannot converge with the larger lag parameters.
            model = ARIMA(history, order=(1,1,0))
            model_fit = model.fit(disp=0)
        output = model_fit.forecast()
        yhat = output[0]

        # Force the result to at least increase by a small value to avoid predictions of all-zeros.
        if yhat[0] < 0.5:
            yhat[0]=0.5

        model_predictions.append(yhat)
        true_test_value = yhat
        # Feedback the result to history for the next prediction
        history.append(true_test_value)

    # Process the results
    M = []
    for m in model_predictions:
        M.append(m.tolist()[0])

    # Restore the values from differences
    training.insert(0, training_data['Deaths'].values[0])   
    for i in range(1, len(training)):
        training[i] = training[i-1] + training[i]
    M[0] = training[-1] + M[0]
    for i in range(1, len(M)):
        M[i] = M[i-1] + M[i]
    print(f"Done: {state}...{M}")
    
    result_death[state] = M[-7:]

Processing: Alabama...
15.966244725738397
Done: Alabama...[3902.9744143363687, 3944.594792020825, 3982.3533408205535, 4014.942213856504, 4046.3898530175093, 4078.6587379549924, 4111.612816810477]
Processing: Alaska...
0.569620253164557
Done: Alaska...[144.93666132979374, 146.47453773935626, 148.01847063974685, 149.55156340378358, 151.09039749571417, 152.63600171789943, 154.1892411769681]
Processing: Arizona...
28.734177215189874
Done: Arizona...[6962.761056156777, 7004.701774690392, 7047.264307163191, 7089.281243047406, 7131.331918239243, 7173.627274415188, 7216.073480043679]
Processing: Arkansas...
10.940928270042194
Done: Arkansas...[2642.7841905843907, 2665.4465628633848, 2688.5747754749955, 2711.816647974534, 2735.14756515728, 2758.5786348424444, 2782.1108320540266]
Processing: California...
81.13924050632912
Done: California...[19909.82647215438, 19982.32593034745, 20094.846507309878, 20238.19453158905, 20393.463027350732, 20575.532070225127, 20715.855282983015]
Processing: Colora

### 3.4 Generate Submission Output File

In [13]:
forecastID = [x for x in range(N_test*50)]
deaths = []
confirmed = []
for i in range(7):
    for s in states:
        print(f"Day {i}/{7}, {s}")
        confirmed.append(result_confirmed[s][i])
        deaths.append(result_death[s][i])


Day 0/7, Alabama
Day 0/7, Alaska
Day 0/7, Arizona
Day 0/7, Arkansas
Day 0/7, California
Day 0/7, Colorado
Day 0/7, Connecticut
Day 0/7, Delaware
Day 0/7, Florida
Day 0/7, Georgia
Day 0/7, Hawaii
Day 0/7, Idaho
Day 0/7, Illinois
Day 0/7, Indiana
Day 0/7, Iowa
Day 0/7, Kansas
Day 0/7, Kentucky
Day 0/7, Louisiana
Day 0/7, Maine
Day 0/7, Maryland
Day 0/7, Massachusetts
Day 0/7, Michigan
Day 0/7, Minnesota
Day 0/7, Mississippi
Day 0/7, Missouri
Day 0/7, Montana
Day 0/7, Nebraska
Day 0/7, Nevada
Day 0/7, New Hampshire
Day 0/7, New Jersey
Day 0/7, New Mexico
Day 0/7, New York
Day 0/7, North Carolina
Day 0/7, North Dakota
Day 0/7, Ohio
Day 0/7, Oklahoma
Day 0/7, Oregon
Day 0/7, Pennsylvania
Day 0/7, Rhode Island
Day 0/7, South Carolina
Day 0/7, South Dakota
Day 0/7, Tennessee
Day 0/7, Texas
Day 0/7, Utah
Day 0/7, Vermont
Day 0/7, Virginia
Day 0/7, Washington
Day 0/7, West Virginia
Day 0/7, Wisconsin
Day 0/7, Wyoming
Day 1/7, Alabama
Day 1/7, Alaska
Day 1/7, Arizona
Day 1/7, Arkansas
Day 1/7, C

In [14]:
round2_path = os.path.join(submissions_dir, 'Team4_Round2.csv')
final = pd.DataFrame(list(zip(forecastID, confirmed, deaths)), 
               columns =['ForecastID', 'Confirmed', 'Deaths']) 
final.to_csv(round2_path, index=False)

# End of Notebook